In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !cp -r -v drive/My\ Drive/ThaiLanguageModel/* .
!rsync -a --progress --exclude="dataset" drive/My\ Drive/ThaiLanguageModel/* .

sending incremental file list
README.md
             19 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=5/6)
character_copus.txt
            259 100%    0.72kB/s    0:00:00 (xfr#2, to-chk=4/6)
data_gen.py
          2,559 100%    3.13kB/s    0:00:00 (xfr#3, to-chk=3/6)
model.ipynb
         74,896 100%   58.09kB/s    0:00:01 (xfr#4, to-chk=2/6)
model.py
              0 100%    0.00kB/s    0:00:00 (xfr#5, to-chk=1/6)
wiki_prepare.py
          2,019 100%    4.59kB/s    0:00:00 (xfr#6, to-chk=0/6)


In [0]:
!rm -rf dataset
!mkdir dataset
!mkdir dataset/thwiki-words
!cp drive/My\ Drive/ThaiLanguageModel/dataset/thwiki-words.zip dataset/thwiki-words.zip
!unzip dataset/thwiki-words.zip -d dataset

In [1]:
!pip install ujson
!pip install pythainlp

     |████████████████████████████████| 194kB 1.1MB/s eta 0:00:01
  Created wheel for ujson: filename=ujson-1.35-cp37-cp37m-linux_x86_64.whl size=69600 sha256=effde6ec631d01ff6dc88b2e273b4d1064dae9e3b031532d02f2fc16ff7f6144
  Stored in directory: /home/porlolicon/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson
     |████████████████████████████████| 11.0MB 12.4MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 10.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/20/95/d23071d0992dabcb61c948fb118a90683193befc88c23e745b050a29e7db/marisa-trie-0.7.5.tar.gz
     |████████████████████████████████| 61kB 12.9MB/s eta 0:00:01
     |████████████████████████████████| 153kB 10.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 11.9MB/s eta 0:00:01
     |████████████████████████████████| 133kB 10.7MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4b

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from keras.layers import CuDNNLSTM, Bidirectional, RepeatVector, Dense, TimeDistributed, Activation, Input
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard
from pathlib import Path
import data_gen

Using TensorFlow backend.


In [7]:
NUM_VOCAB = data_gen.NUM_VOCAB
length = 32
batch_size = 32
train_gen = data_gen.sentence_generator(Path('dataset/thwiki-words-train'),
                                        length,
                                        batch_size,
                                        adversarial=data_gen.random_blank(0.2))
val_gen = data_gen.sentence_generator(Path('dataset/thwiki-words-val'),
                                        length,
                                        batch_size,
                                        adversarial=data_gen.random_blank(0.2))

In [5]:
model = load_model('weight/autoencoder_lstm5/ep002-val_acc:0.26.h5')

W1126 23:54:06.367921 139653896648512 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W1126 23:54:09.511535 139653896648512 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1126 23:54:12.429568 139653896648512 module_wrapper.py:137] From /home/porlolicon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



In [30]:
model = Sequential()
activation = 'relu'
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True), input_shape=(length, NUM_VOCAB)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(64)))
model.add(Activation(activation))

model.add(RepeatVector(length))

model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model.add(Activation(activation))
model.add(TimeDistributed(Dense(NUM_VOCAB)))
model.add(Activation('softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_13 (Bidirectio (None, 32, 256)           221184    
_________________________________________________________________
activation_14 (Activation)   (None, 32, 256)           0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 32, 256)           395264    
_________________________________________________________________
activation_15 (Activation)   (None, 32, 256)           0         
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 32, 128)           164864    
_________________________________________________________________
activation_16 (Activation)   (None, 32, 128)           0         
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 128)              

In [32]:
log_dir = './weight/autoencoder_lstm5'
model.fit_generator(train_gen,
                    steps_per_epoch=4096,
                    validation_data=val_gen,
                    validation_steps=2048,
                    callbacks=[
                        TensorBoard(log_dir),
                        ModelCheckpoint(
                            log_dir + "/ep{epoch:03d}-val_acc:{val_acc:.2f}.h5",
                            monitor='val_acc',
                            mode='max',
                            save_best_only=True,
                        )
                    ],
                    initial_epoch=17,
                    epochs=100)

Epoch 18/100
4096/4096 [==============================] - 140s 34ms/step - loss: 2.8051 - acc: 0.2904 - val_loss: 2.7954 - val_acc: 0.2706
Epoch 19/100
4096/4096 [==============================] - 140s 34ms/step - loss: 2.8131 - acc: 0.2877 - val_loss: 2.8608 - val_acc: 0.2709
Epoch 20/100
4096/4096 [==============================] - 139s 34ms/step - loss: 2.8399 - acc: 0.2818 - val_loss: 2.8585 - val_acc: 0.2728
Epoch 21/100
4096/4096 [==============================] - 140s 34ms/step - loss: 2.8211 - acc: 0.2865 - val_loss: 2.8098 - val_acc: 0.2766
Epoch 22/100
4096/4096 [==============================] - 138s 34ms/step - loss: 2.8303 - acc: 0.2826 - val_loss: 2.7397 - val_acc: 0.2747
Epoch 23/100
 569/4096 [===>..........................] - ETA: 1:39 - loss: 2.8317 - acc: 0.2845

KeyboardInterrupt: 

In [9]:
ads_func = data_gen.random_blank(0.1)

In [31]:
text = 'อยากอกหักแต่อุปสรรคอยู่ที่หน้าตา'
text = ads_func(text)
print('input:\t', text)
encoded = data_gen.encode_text(text, 32)
pred = model.predict(np.expand_dims(encoded, axis=0))[0]
pred_text = data_gen.decode_text(pred)
print('output:\t', pred_text)

input:	 อย$กอกห$กแต่อุปสรรคอยู$ที่หน้าตา
output:	 เางนาาาาาาาาาาาาาาาาาาานาัามี   


In [111]:
len('อยากอกหักแต่อุปสรรคอยู่ที่หน้าตา')

32